## UFC Fighter Prediction project (week 9&10)  

#### This is the first part for this personal project of mine that I have wanted to do for some time. I say first part because there are still many many more things to firs off improve on and secon to expand upon, but this is still a start to that project that will be submitted as my week 9&10 challenge.  

#### The main concept for this project is using extensive and detailed ufc fight datasets, combining it with machine learning models it will make predictions based off of some criteria that I will decide later on but an idea of that could be: predicting low-tier, mid-tier or top-tier fighters or future championship contenders, etc.  

### Making the dataset:  

#### The dataset creation had many parts to it, many loops and ups and downs, it got messy. I found a few different sources for my fight stats, they are the following:  

1: https://github.com/Greco1899/scrape_ufc_stats/tree/main  
2: https://www.kaggle.com/datasets/mdabbert/ultimate-ufc-dataset?resource=download&select=upcoming.csv  
3: https://www.kaggle.com/datasets/asaniczka/ufc-fighters-statistics/data  

#### The reason why this part was tricky and what also made the entire of this project tricky was how the dataset was formed. I will focus on the first two sources as the last one is just a file of fighters' record. Those other datasets are setup in a way that they are divided between fight events, then individual rounds, and then by by red fighter and blue fighter (every ufc fight has a blue and a red corner). It would've been a lot easier if it were setup in a way that each fighter is lined up in a list with all of their corresponding stats. This is one of the improvements I would do to this, and I will eventually, it's to make a better dataset than the one I ended up working with, but for now this is how I set it up: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
path1 = "Data/ufc_fight_stats.csv"
stats1 = pd.read_csv(path1)
df = stats1

In [3]:
df = df.copy()  # Ensure we are working on a fresh copy

# Standardize column names
df.columns = (df.columns.str.strip()
                        .str.replace(' ', '_')
                        .str.replace(r'[^\w]', '', regex=True)  # Remove non-alphanumeric characters
                        .str.upper())

# Function to split "X of Y" format into two columns
def split_x_of_y(column_name, df):
    if column_name in df.columns:
        # Handle missing values gracefully
        df[[f'{column_name}_MADE', f'{column_name}_ATTEMPTED']] = df[column_name].fillna('0 of 0').str.split(' of ', expand=True)
        df[f'{column_name}_MADE'] = pd.to_numeric(df[f'{column_name}_MADE'], errors='coerce').fillna(0).astype(int)
        df[f'{column_name}_ATTEMPTED'] = pd.to_numeric(df[f'{column_name}_ATTEMPTED'], errors='coerce').fillna(0).astype(int)
        df.drop(columns=[column_name], inplace=True)

# Manually define columns to split
cols_to_split = ['SIGSTR', 'TOTALSTR', 'HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND']
for col in cols_to_split:
    split_x_of_y(col, df)

# Convert percentage columns to floats if they exist
percentage_cols = [col for col in df.columns if '%' in col]
for col in percentage_cols:
    df[col] = df[col].str.replace('%', '').astype(float) / 100

# Convert control time (CTRL) to seconds if column exists
def convert_time_to_seconds(time_str):
    if isinstance(time_str, str) and ':' in time_str:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    return np.nan  # Handle missing values

if 'CTRL' in df.columns:
    df['CTRL'] = df['CTRL'].apply(convert_time_to_seconds)

# Convert TD column (Takedown Attempts and Successes) if it exists
if 'TD' in df.columns:
    df[['TD_MADE', 'TD_ATTEMPTED']] = df['TD'].fillna('0 of 0').str.split(' of ', expand=True)
    df['TD_MADE'] = pd.to_numeric(df['TD_MADE'], errors='coerce').fillna(0).astype(int)
    df['TD_ATTEMPTED'] = pd.to_numeric(df['TD_ATTEMPTED'], errors='coerce').fillna(0).astype(int)
    df.drop(columns=['TD'], inplace=True)
    

# Handle missing values
df.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity

In [4]:
df

,EVENT,BOUT,ROUND,FIGHTER,KD,SIGSTR_,TOTAL_STR,TD_,SUBATT,REV,...,LEG_MADE,LEG_ATTEMPTED,DISTANCE_MADE,DISTANCE_ATTEMPTED,CLINCH_MADE,CLINCH_ATTEMPTED,GROUND_MADE,GROUND_ATTEMPTED,TD_MADE,TD_ATTEMPTED
0,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 1,Alex Pereira,0.0,47%,20 of 38,---,0.0,0.0,...,14,18,16,34,0,0,0,0,0,0
1,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 2,Alex Pereira,0.0,50%,18 of 36,---,0.0,0.0,...,10,13,18,36,0,0,0,0,0,0
2,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 3,Alex Pereira,0.0,61%,21 of 34,0%,0.0,0.0,...,15,15,20,33,1,1,0,0,0,1
3,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 4,Alex Pereira,0.0,58%,23 of 29,---,0.0,0.0,...,1,1,1,6,6,6,0,0,0,0
4,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 5,Alex Pereira,0.0,66%,15 of 22,---,0.0,0.0,...,8,8,14,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37697,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0.0,80%,95 of 102,---,0.0,0.0,...,0,0,1,2,2,2,1,1,0,0
37698,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0.0,100%,1 of 1,0%,1.0,0.0,...,0,0,0,0,1,1,0,0,0,1
37699,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0.0,100%,2 of 2,---,0.0,0.0,...,1,1,1,1,0,0,0,0,0,0
37700,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0.0,100%,2 of 2,100%,1.0,0.0,...,0,0,0,0,1,1,0,0,1,1


#### A lot of the stats were show like so: 16 of 34. So for many of the columns I had to split them into two columns, I standardized the column names, converted percentages into floats, converted control time to seconds instead of minutes, and converted the take downs column and lastly handled missing values.  

In [5]:
path2 = "Data/ufc-master.csv"
stats2 = pd.read_csv(path2)
df2 = stats2

In [6]:
# Standardize column names
df2.columns = (df2.columns.str.strip()
                        .str.replace(' ', '_')
                        .str.replace(r'[^\w]', '', regex=True)  # Remove non-alphanumeric characters
                        .str.upper())

# Handle missing values
numeric_cols = df2.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df2.select_dtypes(include=['object']).columns

# Fill missing numerical values with column mean
df2[numeric_cols] = df2[numeric_cols].apply(lambda x: x.fillna(x.mean()))

# Fill missing categorical values with 'Unknown'
df2[categorical_cols] = df2[categorical_cols].fillna('Unknown')

# Convert "FinishRoundTime" (MM:SS) to seconds
def convert_time_to_seconds(time_str):
    if isinstance(time_str, str) and ':' in time_str:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    return np.nan

if 'FINISHROUNDTIME' in df2.columns:
    df2['FINISHROUNDTIME_SECS'] = df2['FINISHROUNDTIME'].apply(convert_time_to_seconds)
    df2.drop(columns=['FINISHROUNDTIME'], inplace=True)

# Standardize fighter stances
stance_mapping = {
    'Orthodox': 'Orthodox',
    'orthodox': 'Orthodox',
    'Southpaw': 'Southpaw',
    'southpaw': 'Southpaw',
    'Switch': 'Switch',
    'switch': 'Switch',
    'Unknown': 'Unknown'
}
if 'REDSTANCE' in df2.columns:
    df2['REDSTANCE'] = df2['REDSTANCE'].map(stance_mapping).fillna('Unknown')
if 'BLUESTANCE' in df2.columns:
    df2['BLUESTANCE'] = df2['BLUESTANCE'].map(stance_mapping).fillna('Unknown')

# Normalize Red vs Blue Differences
if all(col in df2.columns for col in ['REDWINS', 'BLUEWINS']):
    df2['WINDIF'] = df2['REDWINS'] - df2['BLUEWINS']

if all(col in df2.columns for col in ['REDREACHCMS', 'BLUEREACHCMS', 'REDHEIGHTCMS', 'BLUEHEIGHTCMS', 'REDAGE', 'BLUEAGE']):
    df2['REACHDIF'] = df2['REDREACHCMS'] - df2['BLUEREACHCMS']
    df2['HEIGHTDIF'] = df2['REDHEIGHTCMS'] - df2['BLUEHEIGHTCMS']
    df2['AGEDIF'] = df2['REDAGE'] - df2['BLUEAGE']

C:\Users\emilp\AppData\Local\Temp\ipykernel_9704\4136878062.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2['FINISHROUNDTIME_SECS'] = df2['FINISHROUNDTIME'].apply(convert_time_to_seconds)


In [7]:
df2

,REDFIGHTER,BLUEFIGHTER,REDODDS,BLUEODDS,REDEXPECTEDVALUE,BLUEEXPECTEDVALUE,DATE,LOCATION,COUNTRY,WINNER,...,FINISHDETAILS,FINISHROUND,TOTALFIGHTTIMESECS,REDDECODDS,BLUEDECODDS,RSUBODDS,BSUBODDS,RKOODDS,BKOODDS,FINISHROUNDTIME_SECS
0,Alexandre Pantoja,Kai Asakura,-250.0,215.0,40.0000,215.0,07/12/2024,"Las Vegas, Nevada, USA",USA,Red,...,Rear Naked Choke,2.0,425.0,300.000000,800.000000,150.000000,2500.000000,400.000000,350.000000,125.0
1,Shavkat Rakhmonov,Ian Machado Garry,-210.0,295.0,47.6190,295.0,07/12/2024,"Las Vegas, Nevada, USA",USA,Red,...,Unknown,5.0,1500.0,250.000000,650.000000,180.000000,3000.000000,240.000000,700.000000,300.0
2,Ciryl Gane,Alexander Volkov,-380.0,300.0,26.3158,300.0,07/12/2024,"Las Vegas, Nevada, USA",USA,Red,...,Unknown,3.0,900.0,-160.000000,450.000000,1100.000000,3000.000000,350.000000,1100.000000,300.0
3,Bryce Mitchell,Kron Gracie,-950.0,625.0,10.5263,625.0,07/12/2024,"Las Vegas, Nevada, USA",USA,Red,...,Elbows,3.0,639.0,-200.000000,1100.000000,380.000000,1400.000000,500.000000,4000.000000,39.0
4,Nate Landwehr,Dooho Choi,-130.0,110.0,76.9231,110.0,07/12/2024,"Las Vegas, Nevada, USA",USA,Blue,...,Elbows,3.0,801.0,275.000000,550.000000,500.000000,700.000000,300.000000,250.000000,201.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6523,Duane Ludwig,Darren Elkins,-155.0,135.0,64.5161,135.0,21/03/2010,"Broomfield, Colorado, USA",USA,Blue,...,Unknown,1.0,44.0,308.333395,425.870288,884.048151,1100.497775,510.891606,636.463235,44.0
6524,John Howard,Daniel Roberts,-210.0,175.0,47.6190,175.0,21/03/2010,"Broomfield, Colorado, USA",USA,Red,...,Punch,1.0,121.0,308.333395,425.870288,884.048151,1100.497775,510.891606,636.463235,121.0
6525,Brendan Schaub,Chase Gormley,-260.0,220.0,38.4615,220.0,21/03/2010,"Broomfield, Colorado, USA",USA,Red,...,Punches,1.0,47.0,308.333395,425.870288,884.048151,1100.497775,510.891606,636.463235,47.0
6526,Mike Pierce,Julio Paulino,-420.0,335.0,23.8095,335.0,21/03/2010,"Broomfield, Colorado, USA",USA,Red,...,Unknown,3.0,900.0,308.333395,425.870288,884.048151,1100.497775,510.891606,636.463235,300.0


#### For this one, aside from applying the same methods as with the last dataset, I standardized the figting stances column and normalized the red vs blue differences(win diff, reach diff, height diff, etc) 

In [ ]:
df['FIGHTER'] = df['FIGHTER'].str.strip().str.lower()
df2['REDFIGHTER'] = df2['REDFIGHTER'].str.strip().str.lower()
df2['BLUEFIGHTER'] = df2['BLUEFIGHTER'].str.strip().str.lower()

# Merge on REDFIGHTER
merged_red = df.merge(df2, left_on='FIGHTER', right_on='REDFIGHTER', how='left', suffixes=('', '_RED'))

# Merge on BLUEFIGHTER
merged_blue = df.merge(df2, left_on='FIGHTER', right_on='BLUEFIGHTER', how='left', suffixes=('', '_BLUE'))

# Combine both merged DataFrames
df_combined = pd.concat([merged_red, merged_blue], ignore_index=True)

# Remove duplicated columns, if any
df_combined = df_combined.loc[:, ~df_combined.columns.duplicated()]

df_combined.fillna(0, inplace=True)

# Extract the number from 'Round X' safely, keeping NaNs intact
df_combined['ROUND'] = df_combined['ROUND'].str.extract(r'(\d+)')  # Extract digits
df_combined['ROUND'] = pd.to_numeric(df_combined['ROUND'], errors='coerce')  # Convert to numeric

# Fill NaNs with 0 or any default value you want, then convert to int
df_combined['ROUND'] = df_combined['ROUND'].fillna(0).astype(int)

# Save or preview
df_combined.to_csv("Data/combined_ufc_dataset_2.csv", index=False)
print("Combined dataset shape:", df_combined.shape)
print(df_combined.head())

#### And over here is where I combine both datasets based on the fighter names

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv("Data/ufc-master.csv")  # Adjust path if needed

# Normalize the fighter names to ensure correct capitalization
df['RedFighter'] = df['RedFighter'].str.strip().str.title()  # Normalize Red fighter names
df['BlueFighter'] = df['BlueFighter'].str.strip().str.title()  # Normalize Blue fighter names

# Replace the 'Red' and 'Blue' columns with the actual fighter names
df['Fighter'] = df['RedFighter']
df['Opponent'] = df['BlueFighter']

# Remove the original 'RedFighter' and 'BlueFighter' columns
df = df.drop(columns=['RedFighter', 'BlueFighter'])

# Convert the 'Date' column to a datetime format for easier handling
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Add separate date columns for RedFighter and BlueFighter
df['RedDate'] = df['Date']
df['BlueDate'] = df['Date']

# Calculate the latest fight date for each fighter (whether Red or Blue)
latest_dates = pd.concat([df[['Fighter', 'RedDate']], df[['Fighter', 'BlueDate']]], axis=0)

# Drop duplicate rows to ensure each fighter has the most recent fight date
latest_dates = latest_dates.groupby('Fighter').max().reset_index()

# Merge the latest fight dates into the main dataframe
df = df.merge(latest_dates, on='Fighter', how='left')

# List of columns that need to be aggregated (both numeric and categorical)
columns_to_aggregate = [
    'RedOdds', 'BlueOdds', 'RedExpectedValue', 'BlueExpectedValue', 'Winner', 'TitleBout', 'WeightClass', 
    'Gender', 'NumberOfRounds', 'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws', 
    'BlueAvgSigStrLanded', 'BlueAvgSigStrPct', 'BlueAvgSubAtt', 'BlueAvgTDLanded', 'BlueAvgTDPct', 
    'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought', 'BlueTotalTitleBouts', 
    'BlueWinsByDecisionMajority', 'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous', 
    'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage', 'BlueWins', 'BlueStance', 
    'BlueHeightCms', 'BlueReachCms', 'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak', 
    'RedDraws', 'RedAvgSigStrLanded', 'RedAvgSigStrPct', 'RedAvgSubAtt', 'RedAvgTDLanded', 'RedAvgTDPct', 
    'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought', 'RedTotalTitleBouts', 
    'RedWinsByDecisionMajority', 'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO', 
    'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins', 'RedStance', 'RedHeightCms', 
    'RedReachCms', 'RedWeightLbs', 'LoseStreakDif', 'WinStreakDif', 'LongestWinStreakDif', 
    'WinDif', 'LossDif', 'TotalRoundDif', 'TotalTitleBoutDif', 'KODif', 'SubDif', 'HeightDif', 'ReachDif', 
    'AgeDif', 'SigStrDif', 'AvgSubAttDif', 'AvgTDDif', 'EmptyArena', 'BMatchWCRank', 'RMatchWCRank', 
    'RWFlyweightRank', 'RWFeatherweightRank', 'RWStrawweightRank', 'RWBantamweightRank', 'RHeavyweightRank', 
    'RLightHeavyweightRank', 'RMiddleweightRank', 'RWelterweightRank', 'RLightweightRank', 'RFeatherweightRank', 
    'RBantamweightRank', 'RFlyweightRank', 'RPFPRank', 'BWFlyweightRank', 'BWFeatherweightRank', 'BWStrawweightRank', 
    'BWBantamweightRank', 'BHeavyweightRank', 'BLightHeavyweightRank', 'BMiddleweightRank', 'BWelterweightRank', 
    'BLightweightRank', 'BFeatherweightRank', 'BBantamweightRank', 'BFlyweightRank', 'BPFPRank', 'BetterRank', 
    'Finish', 'FinishDetails', 'FinishRound', 'FinishRoundTime', 'TotalFightTimeSecs', 'RedDecOdds', 'BlueDecOdds', 
    'RSubOdds', 'BSubOdds', 'RKOOdds', 'BKOOdds', 'Age' 
]

# Filter columns to only include numeric columns for aggregation
numeric_columns = df[columns_to_aggregate].select_dtypes(include=[np.number]).columns.tolist()

# Ensure all the columns to aggregate are numeric
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Aggregate the statistics by averaging the values for each fighter
fighter_stats = df.groupby('Fighter')[numeric_columns].mean().reset_index()

# Normalize the features (except 'Age' and 'Wins')
scaler = MinMaxScaler()

# Remove 'Age' and 'Wins' from normalization
numeric_columns_without_age_wins = [col for col in numeric_columns if col not in ['Age', 'Wins']]

# Normalize all numeric columns except 'Age' and 'Wins'
fighter_stats[numeric_columns_without_age_wins] = scaler.fit_transform(fighter_stats[numeric_columns_without_age_wins])

# Add the latest date for each fighter
fighter_stats = fighter_stats.merge(latest_dates[['Fighter', 'RedDate']], on='Fighter', how='left')

# Save the final dataset to CSV
fighter_stats.to_csv("Data/fighter_fpi_normalized_2.csv", index=False)

# Display the top fighters
print(fighter_stats)


#### This last code replaces red and blue columns with fighter names, finding the latest date for a fighter to keep track of active and inactive fighters, then I averaged out all of the fight stats that were aggregated and normalized them.  

### The next part of the project is the modeling and training.  

#### It's split into two parts, one is the making of the actual model where all the predictions happen and the second is the making of an ILO score for each ufc fighter. We will start with the second one.  

### Creating ILO scores:  

#### This ILO score stands for Intelligent Operator Score, a measure we will use to assess a fighter's overall abilities. It combines an ELO rating and an FPI score, we will start with ELO. An ELO rating was first used in professional chess and then later used for other sports to measure an individual or teams abilities and ranking. After each match the score is updated, the winner gains points and the loser loses points. The loser always loses the same amount the winner gets. This is the base of an Elo score, one of the things we will use for the project:

In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("Data/combined_ufc_dataset.csv")

# Remove exact duplicates
df = df.drop_duplicates()

# Create sorted versions of fighters to identify duplicate matchups
df['F1'] = df[['REDFIGHTER', 'BLUEFIGHTER']].min(axis=1)
df['F2'] = df[['REDFIGHTER', 'BLUEFIGHTER']].max(axis=1)

# Remove mirrored/repeated matchups
df = df.drop_duplicates(subset=['F1', 'F2', 'WINNER'])

# Drop helper columns
df.drop(columns=['F1', 'F2'], inplace=True)

# Initialize Elo ratings
initial_elo = 1000
elo_ratings = {}
K = 32  # Standard Elo sensitivity

# Expected outcome function
def expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

# Update Elo ratings without bonuses
def update_elo(winner, loser):
    if winner not in elo_ratings:
        elo_ratings[winner] = initial_elo
    if loser not in elo_ratings:
        elo_ratings[loser] = initial_elo

    Ra = elo_ratings[winner]
    Rb = elo_ratings[loser]

    Ea = expected_score(Ra, Rb)
    Eb = expected_score(Rb, Ra)

    Ra_new = Ra + K * (1 - Ea)
    Rb_new = Rb + K * (0 - Eb)

    elo_ratings[winner] = Ra_new
    elo_ratings[loser] = Rb_new

# Process fights chronologically
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
df = df.dropna(subset=['DATE'])
df = df.sort_values('DATE')

# Loop over fights and update Elo
for _, row in df.iterrows():
    red = str(row['REDFIGHTER']).strip()
    blue = str(row['BLUEFIGHTER']).strip()
    winner_color = str(row['WINNER']).strip().upper()

    if winner_color == 'RED':
        winner = red
        loser = blue
    elif winner_color == 'BLUE':
        winner = blue
        loser = red
    else:
        continue  # Skip if draw or invalid

    update_elo(winner, loser)

# Convert Elo ratings to DataFrame
elo_df = pd.DataFrame.from_dict(elo_ratings, orient='index', columns=['ELO'])
elo_df.reset_index(inplace=True)
elo_df.rename(columns={'index': 'FIGHTER'}, inplace=True)

elo_df.to_csv("Data/elo_ratings.csv", index=False)

# Show top 10 fighters
print(elo_df.sort_values(by='ELO', ascending=False).head(10))

C:\Users\emilp\AppData\Local\Temp\ipykernel_9704\319916400.py:4: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/combined_ufc_dataset.csv")


                    FIGHTER          ELO
397            amanda nunes  1122.588021
146          dustin poirier  1115.035133
12                jon jones  1114.859447
301            max holloway  1110.859075
1025        israel adesanya  1109.234375
8          charles oliveira  1102.713350
900           sean o'malley  1100.448669
987   alexander volkanovski  1094.259399
1266      dricus du plessis  1093.942097
869            shane burgos  1087.039414


#### It first normalizes and cleans the data, removing any duplicates and creating new features like the "Wins" column (the higher of the Red and Blue fighter's wins). Then, the code iterates through each fight in chronological order, updates the Elo ratings for both the winner and the loser using the standard Elo formula, and handles ties or invalid results by skipping them. It also merges the latest fight date for each fighter and calculates various fight statistics. After all fights are processed, the Elo ratings are stored in a DataFrame and saved to a CSV file. Finally, the top 10 fighters based on their Elo ratings are displayed.  

#### Next we will develop the FPI scores which stands for Fighter Performance Index which uses weighted values for chosen features to calculate the score:

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load dataset
df = pd.read_csv("Data/fighter_fpi_normalized.csv")

# Step 1: Convert 'RedDate' to datetime
df['RedDate'] = pd.to_datetime(df['RedDate'], format='%d/%m/%Y', errors='coerce')

# Step 2: Calculate the inactivity period (in years) based on RedDate
current_date = pd.to_datetime('today')
df['INACTIVITY_PERIOD'] = (current_date - df['RedDate']).dt.days / 365  # In years

# Step 3: Filter out fighters who have been inactive for more than 4 years
active_fighters_df = df[df['INACTIVITY_PERIOD'] <= 3]

# Step 4: Define safe division to avoid division by zero
def safe_div(a, b):
    return np.where(b == 0, 0, a / b)

# Step 5: Compute individual fighter statistics (no "Red" or "Blue" dynamic)
active_fighters_df['SIGSTR_ACC'] = safe_div(active_fighters_df['RedAvgSigStrLanded'] + active_fighters_df['BlueAvgSigStrLanded'], active_fighters_df['RedAvgSigStrPct'] + active_fighters_df['BlueAvgSigStrPct'])
active_fighters_df['TD_ACC'] = safe_div(active_fighters_df['RedAvgTDLanded'] + active_fighters_df['BlueAvgTDLanded'], active_fighters_df['RedAvgTDPct'] + active_fighters_df['BlueAvgTDPct'])
active_fighters_df['FINISH_RATE'] = safe_div(
    active_fighters_df['RedWinsByKO'] + active_fighters_df['BlueWinsByKO'] + active_fighters_df['RedWinsBySubmission'] + active_fighters_df['BlueWinsBySubmission'],
    active_fighters_df['RedWins'] + active_fighters_df['BlueWins'] + active_fighters_df['RedLosses'] + active_fighters_df['BlueLosses']
)

# Step 6: Calculate opponent quality (combine Red and Blue opponents)
active_fighters_df['OPP_QUALITY'] = (active_fighters_df['RedWins'] + active_fighters_df['BlueWins']) / (active_fighters_df['RedWins'] + active_fighters_df['BlueWins'] + active_fighters_df['RedLosses'] + active_fighters_df['BlueLosses'] + 1)

# Step 7: Define FPI features (combining Red and Blue fighter stats)
fpi_features = [
    'RedAvgSigStrLanded', 'BlueAvgSigStrLanded', 'RedAvgSubAtt', 'BlueAvgSubAtt', 'RedAvgTDLanded', 'BlueAvgTDLanded',
    'SIGSTR_ACC', 'TD_ACC', 'RedHeightCms', 'BlueHeightCms', 'RedReachCms', 'BlueReachCms', 'RedWins', 'BlueWins',
    'RedLongestWinStreak', 'BlueLongestWinStreak', 'RedTotalTitleBouts', 'BlueTotalTitleBouts', 'FINISH_RATE', 'OPP_QUALITY'
]

# Step 8: Remove NaNs for clean normalization
active_fighters_df = active_fighters_df.dropna(subset=fpi_features)

# Step 9: Normalize features (for all numeric features)
scaler = MinMaxScaler()

# Apply normalization to the features
active_fighters_df[fpi_features] = scaler.fit_transform(active_fighters_df[fpi_features])

# Step 10: Define feature weights (higher weight for more important features)
weights = {
    'RedAvgSigStrLanded': 1.2,
    'BlueAvgSigStrLanded': 1.2,
    'RedAvgSubAtt': 1.0,
    'BlueAvgSubAtt': 1.0,
    'RedAvgTDLanded': 1.1,
    'BlueAvgTDLanded': 1.1,
    'SIGSTR_ACC': 1.5,
    'TD_ACC': 1.3,
    'RedHeightCms': 0.1,
    'BlueHeightCms': 0.1,
    'RedReachCms': 0.1,
    'BlueReachCms': 0.1,
    'RedWins': 1.4,
    'BlueWins': 1.4,
    'RedLongestWinStreak': 0.9,
    'BlueLongestWinStreak': 0.9,
    'RedTotalTitleBouts': 0.5,
    'BlueTotalTitleBouts': 0.5,
    'FINISH_RATE': 1.4,
    'OPP_QUALITY': 2
}

# Step 11: Weighted FPI calculation for each fighter
active_fighters_df['FPI'] = active_fighters_df.apply(lambda row: sum(row[feature] * weights.get(feature, 1) for feature in fpi_features), axis=1)

# Step 12: Aggregate stats to fighter level (average over all available fights)
fpi_df = active_fighters_df.groupby('Fighter')['FPI'].mean().reset_index()

# Step 13: Save FPI scores to CSV
fpi_df.to_csv("Data/fighter_fpi_scores_active.csv", index=False)

# Step 14: Display top 10 fighters by FPI
print(fpi_df.sort_values(by='FPI', ascending=False).head(20))


                   Fighter       FPI
297              Jon Jones  9.527688
302              Jose Aldo  9.350019
415           Max Holloway  8.605211
498       Rafael Dos Anjos  8.429580
518          Robbie Lawler  8.240213
114          Chris Weidman  8.143888
328           Kamaru Usman  7.866836
26       Aljamain Sterling  7.677319
22   Alexander Volkanovski  7.595881
174      Dricus Du Plessis  7.564885
214        Glover Teixeira  7.504076
590          Tony Ferguson  7.486631
347             King Green  7.465324
176         Dustin Poirier  7.437736
105       Charles Oliveira  7.427149
412           Mauricio Rua  7.402977
342              Kevin Lee  7.397354
456              Nate Diaz  7.354178
531         Rose Namajunas  7.275599
563       Stephen Thompson  7.272934


C:\Users\emilp\AppData\Local\Temp\ipykernel_9704\422572269.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_fighters_df['SIGSTR_ACC'] = safe_div(active_fighters_df['RedAvgSigStrLanded'] + active_fighters_df['BlueAvgSigStrLanded'], active_fighters_df['RedAvgSigStrPct'] + active_fighters_df['BlueAvgSigStrPct'])
C:\Users\emilp\AppData\Local\Temp\ipykernel_9704\422572269.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_fighters_df['TD_ACC'] = safe_div(active_fighters_df['RedAvgTDLanded'] +

#### The code processes a UFC dataset to compute fighter performance indices (FPIs) for active fighters based on various fight statistics. It first normalizes the fighter names and calculates the inactivity period for each fighter based on the date of their last fight. Fighters inactive for more than four years are filtered out. The code then computes statistics such as strike accuracy, takedown accuracy, and finish rate for both red and blue fighters in each fight. It calculates the quality of each fighter's opponents and normalizes the statistics using MinMax scaling. Next, a weighted FPI is calculated for both red and blue fighters, with the weight for each feature being customizable. The FPIs are aggregated for each fighter, and the final results are saved to a CSV file. The top 10 active fighters are displayed based on their FPI.  

#### Then I merged both scores to make an ILO score:

In [ ]:

elo_df = pd.read_csv("Data/elo_ratings.csv")  # Elo scores
fpi_df = pd.read_csv("Data/fighter_fpi_scores_active.csv")  # FPI scores

# Merge the Elo and FPI scores on the 'FIGHTER' column
merged_df = pd.merge(elo_df, fpi_df, on='FIGHTER', how='inner')

# Normalize Elo and FPI scores to the range [0, 1]
merged_df['Elo_normalized'] = (merged_df['ELO'] - merged_df['ELO'].min()) / (merged_df['ELO'].max() - merged_df['ELO'].min())
merged_df['FPI_normalized'] = (merged_df['FPI'] - merged_df['FPI'].min()) / (merged_df['FPI'].max() - merged_df['FPI'].min())

#Define the weights for Elo and FPI
Weight_Elo = 0.5
Weight_FPI = 0.5

#  Calculate ILO score (weighted average of Elo and FPI)
merged_df['ILO'] = (Weight_Elo * merged_df['Elo_normalized']) + (Weight_FPI * merged_df['FPI_normalized'])

# Save ILO scores to CSV
merged_df.to_csv("Data/fighter_ilo_scores.csv", index=False)

# Display top 10 fighters by ILO
print(merged_df.sort_values(by='ILO', ascending=False).head(20))